In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# --- Import Library ---
import pandas as pd
from statsmodels.stats.contingency_tables import mcnemar
from IPython.display import display

In [ ]:
# --- Load hasil klasifikasi ---
df_bg = pd.read_csv('/content/drive/MyDrive/HANA/hasil_klasifikasi_background.csv')
df_nobg = pd.read_csv('/content/drive/MyDrive/HANA/hasil_klasifikasi_hapus_background.csv')

# --- Pastikan urutan filename sama ---
df_bg_sorted = df_bg.sort_values(by='Filename').reset_index(drop=True)
df_nobg_sorted = df_nobg.sort_values(by='Filename').reset_index(drop=True)

# --- Gabungkan hasil prediksi untuk perbandingan ---
evaluasi_df = pd.DataFrame({
    'Filename': df_bg_sorted['Filename'],
    'Label Asli': df_bg_sorted['Label Aktual'],
    'Prediksi_Background': df_bg_sorted['Label Prediksi'],
    'Prediksi_NoBackground': df_nobg_sorted['Label Prediksi']
})

# --- Cek kebenaran masing-masing model ---
evaluasi_df['BG_Benar?'] = evaluasi_df['Label Asli'] == evaluasi_df['Prediksi_Background']
evaluasi_df['NoBG_Benar?'] = evaluasi_df['Label Asli'] == evaluasi_df['Prediksi_NoBackground']

In [ ]:
# --- Tentukan kategori McNemar ---
def tentukan_kategori(row):
    if row['NoBG_Benar?'] and not row['BG_Benar?']:
        return 'b (NoBG benar, BG salah)'
    elif not row['NoBG_Benar?'] and row['BG_Benar?']:
        return 'c (NoBG salah, BG benar)'
    else:
        return '-'

evaluasi_df['Kategori McNemar'] = evaluasi_df.apply(tentukan_kategori, axis=1)

# --- Buat tabel kontingensi McNemar ---
b = ((evaluasi_df['NoBG_Benar?'] == True) & (evaluasi_df['BG_Benar?'] == False)).sum()
c = ((evaluasi_df['NoBG_Benar?'] == False) & (evaluasi_df['BG_Benar?'] == True)).sum()

print(f"Nilai b (NoBG benar, BG salah): {b}")
print(f"Nilai c (NoBG salah, BG benar): {c}")

# --- Uji McNemar ---
table = [[0, b],
         [c, 0]]

hasil_mcnemar = mcnemar(table, exact=True)  # gunakan exact=True karena sampel kecil

print("\n📊 Hasil Uji McNemar:")
print(f"Statistik: {hasil_mcnemar.statistic}")
print(f"p-value  : {hasil_mcnemar.pvalue}")

# --- Interpretasi ---
alpha = 0.05
if hasil_mcnemar.pvalue < alpha:
    print("\n❗ Terdapat perbedaan signifikan antara model dengan dan tanpa background (tolak H0)")
else:
    print("\n✅ Tidak terdapat perbedaan signifikan antara model dengan dan tanpa background (gagal tolak H0)")

# --- Tampilkan Tabel Evaluasi Lengkap ---
print("\n📋 Tabel Evaluasi Lengkap:")
display(evaluasi_df)

# --- Simpan ke Excel (opsional) ---
evaluasi_df.to_excel('/content/drive/MyDrive/HANA/Uji_McNemar_Compare_Background_vs_NoBackground.xlsx', index=False)
print("\n✅ Tabel evaluasi disimpan ke Excel.")

Nilai b (NoBG benar, BG salah): 0
Nilai c (NoBG salah, BG benar): 38

📊 Hasil Uji McNemar:
Statistik: 0.0
p-value  : 7.275957614183426e-12

❗ Terdapat perbedaan signifikan antara model dengan dan tanpa background (tolak H0)

📋 Tabel Evaluasi Lengkap:


,Filename,Label Asli,Prediksi_Background,Prediksi_NoBackground,BG_Benar?,NoBG_Benar?,Kategori McNemar
0,masak (13).JPG,Masak(b),Mengkal(b),Masak,False,False,-
1,masak (15).JPG,Masak(b),Mengkal(b),Masak,False,False,-
2,masak (2).JPG,Masak(b),Mengkal(b),Mentah,False,False,-
3,masak (24).JPG,Masak(b),Mengkal(b),Mentah,False,False,-
4,masak (26).JPG,Masak(b),Mengkal(b),Mentah,False,False,-
5,masak (27).JPG,Masak(b),Mentah(b),Masak,False,False,-
6,masak (30).jpg,Masak(b),Masak(b),Mentah,True,False,"c (NoBG salah, BG benar)"
7,masak (32).jpg,Masak(b),Masak(b),Masak,True,False,"c (NoBG salah, BG benar)"
8,masak (33).jpg,Masak(b),Mengkal(b),Mentah,False,False,-
9,masak (38).jpg,Masak(b),Masak(b),Masak,True,False,"c (NoBG salah, BG benar)"



✅ Tabel evaluasi disimpan ke Excel.


In [ ]:
from statsmodels.stats.contingency_tables import mcnemar
import numpy as np

# Misalnya hasil prediksi model SVM dan Naive Bayes yang sudah ada
# Gantilah dengan prediksi yang sesuai dari model Anda
# svm_pred_with_bg = model_svm_with_bg.predict(X_test)
# svm_pred_without_bg = model_svm_without_bg.predict(X_test)
# nb_pred_with_bg = model_nb_with_bg.predict(X_test)
# nb_pred_without_bg = model_nb_without_bg.predict(X_test)

# Gunakan data dari evaluasi_df untuk label aktual dan prediksi
y_test = evaluasi_df['Label Asli']
svm_pred_with_bg = evaluasi_df['Prediksi_Background']
svm_pred_without_bg = evaluasi_df['Prediksi_NoBackground']

# Dengan asumsi kolom 'Prediksi_Background' dan 'Prediksi_NoBackground' merepresentasikan prediksi SVM
# Jika kolom-kolom ini merepresentasikan prediksi dari model yang berbeda (misalnya, SVM dan Naive Bayes),
# Anda perlu menyesuaikan penugasan variabel.
# Misalnya, jika 'Prediksi_Background' adalah SVM dan 'Prediksi_NoBackground' adalah Naive Bayes:
# svm_pred_with_bg = evaluasi_df['Prediksi_Background']
# nb_pred_without_bg = evaluasi_df['Prediksi_NoBackground']

# Untuk uji McNemar ini membandingkan model yang *sama* dengan dan tanpa background,
# kita akan menggunakan prediksi dari kedua skenario. Dengan asumsi 'Prediksi_Background'
# dan 'Prediksi_NoBackground' berasal dari model yang *sama*.
# Jika Anda membandingkan model yang berbeda, Anda perlu memuat prediksi tersebut.

# Mari kita asumsikan 'Prediksi_Background' dan 'Prediksi_NoBackground' berasal dari model yang sama.
# Kita membandingkan performa model tunggal ini *dengan* dan *tanpa* penghapusan background.

# Buat vektor biner: 1 jika prediksi benar, 0 jika salah untuk model dengan dan tanpa background
correct_with_bg = (svm_pred_with_bg == y_test)
correct_without_bg = (svm_pred_without_bg == y_test)


# Hitung b dan c untuk model dengan dan tanpa penghapusan background
b = np.sum((correct_without_bg == True) & (correct_with_bg == False))
c = np.sum((correct_without_bg == False) & (correct_with_bg == True))

# Buat tabel kontingensi 2x2 untuk perbandingan
table = [[0, b],
         [c, 0]]

# Uji McNemar
hasil_mcnemar = mcnemar(table, exact=True)

# Tampilkan hasil uji McNemar
print(f"b (NoBG benar, BG salah) = {b}, c (NoBG salah, BG benar) = {c}")
print(f"Statistik: {hasil_mcnemar.statistic}, p-value: {hasil_mcnemar.pvalue:.10f}") # Tingkatkan presisi p-value

# Interpretasi
alpha = 0.05
if hasil_mcnemar.pvalue < alpha:
    print("✅ Hasil uji McNemar menunjukkan bahwa perbedaan performa antara skenario dengan dan tanpa penghapusan latar belakang SIGNIFIKAN secara statistik (p-value < 0.05).")
else:
    print("❌ Hasil uji McNemar menunjukkan bahwa perbedaan performa antara skenario dengan dan tanpa penghapusan latar belakang TIDAK signifikan secara statistik (p-value ≥ 0.05).")

b (NoBG benar, BG salah) = 0, c (NoBG salah, BG benar) = 38
Statistik: 0.0, p-value: 0.0000000000
✅ Hasil uji McNemar menunjukkan bahwa perbedaan performa antara skenario dengan dan tanpa penghapusan latar belakang SIGNIFIKAN secara statistik (p-value < 0.05).
